### User:

Can you walk me through the following code?

In [ ]:
import dash
from dash.dependencies import Input, Output, State
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_table
import plotly.express as px
import re
import time
from skimage import io

DEBUG = True

NUM_ATYPES = 15
DEFAULT_FIG_MODE = "layout"
annotation_colormap = px.colors.qualitative.Light24
annotation_types = [
    "tree",
    "building",
    "sky",
    "road",
    "sidewalk",
    "car",
    "pedestrian",
    "cyclist",
    "stop sign",
    "parking sign",
    "traffic light",
    "lamp post",
    "star",  # e.g., sun or moon as to not confuse them with artificial lighting
]
DEFAULT_ATYPE = annotation_types[0]

# prepare bijective type<->color mapping
typ_col_pairs = [
    (t, annotation_colormap[n % len(annotation_colormap)])
    for n, t in enumerate(annotation_types)
]
# types to colors
color_dict = {}
# colors to types
type_dict = {}
for typ, col in typ_col_pairs:
    color_dict[typ] = col
    type_dict[col] = typ

options = list(color_dict.keys())
columns = ["Type", "X0", "Y0", "X1", "Y1"]
# Open the readme for use in the context info
with open("assets/Howto.md", "r") as f:
    # Using .read rather than .readlines because dcc.Markdown
    # joins list of strings with newline characters
    howto = f.read()


def debug_print(*args):
    if DEBUG:
        print(*args)


def coord_to_tab_column(coord):
    return coord.upper()


def time_passed(start=0):
    return round(time.mktime(time.localtime())) - start


def format_float(f):
    return "%.2f" % (float(f),)


def shape_to_table_row(sh):
    return {
        "Type": type_dict[sh["line"]["color"]],
        "X0": format_float(sh["x0"]),
        "Y0": format_float(sh["y0"]),
        "X1": format_float(sh["x1"]),
        "Y1": format_float(sh["y1"]),
    }


def default_table_row():
    return {
        "Type": DEFAULT_ATYPE,
        "X0": format_float(10),
        "Y0": format_float(10),
        "X1": format_float(20),
        "Y1": format_float(20),
    }


def table_row_to_shape(tr):
    return {
        "editable": True,
        "xref": "x",
        "yref": "y",
        "layer": "above",
        "opacity": 1,
        "line": {"color": color_dict[tr["Type"]], "width": 4, "dash": "solid"},
        "fillcolor": "rgba(0, 0, 0, 0)",
        "fillrule": "evenodd",
        "type": "rect",
        "x0": tr["X0"],
        "y0": tr["Y0"],
        "x1": tr["X1"],
        "y1": tr["Y1"],
    }


def shape_cmp(s0, s1):
    """ Compare two shapes """
    return (
        (s0["x0"] == s1["x0"])
        and (s0["x1"] == s1["x1"])
        and (s0["y0"] == s1["y0"])
        and (s0["y1"] == s1["y1"])
        and (s0["line"]["color"] == s1["line"]["color"])
    )


def shape_in(se):
    """ check if a shape is in list (done this way to use custom compare) """
    return lambda s: any(shape_cmp(s, s_) for s_ in se)


def index_of_shape(shapes, shape):
    for i, shapes_item in enumerate(shapes):
        if shape_cmp(shapes_item, shape):
            return i
    raise ValueError  # not found


def annotations_table_shape_resize(annotations_table_data, fig_data):
    """
    Extract the shape that was resized (its index) and store the resized
    coordinates.
    """
    debug_print("fig_data", fig_data)
    debug_print("table_data", annotations_table_data)
    for key, val in fig_data.items():
        shape_nb, coord = key.split(".")
        # shape_nb is for example 'shapes[2].x0': this extracts the number
        shape_nb = shape_nb.split(".")[0].split("[")[-1].split("]")[0]
        # this should correspond to the same row in the data table
        # we have to format the float here because this is exactly the entry in
        # the table
        annotations_table_data[int(shape_nb)][
            coord_to_tab_column(coord)
        ] = format_float(fig_data[key])
        # (no need to compute a time stamp, that is done for any change in the
        # table values, so will be done later)
    return annotations_table_data


def shape_data_remove_timestamp(shape):
    """
    go.Figure complains if we include the 'timestamp' key when updating the
    figure
    """
    new_shape = dict()
    for k in shape.keys() - set(["timestamp"]):
        new_shape[k] = shape[k]
    return new_shape


external_stylesheets = [dbc.themes.BOOTSTRAP, "assets/image_annotation_style.css"]
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

filelist = [
    "assets/driving.jpg",
    "assets/professional-transport-autos-bridge-traffic-road-rush-hour.jpg",
    "assets/rocket.jpg",
]

server = app.server

fig = px.imshow(io.imread(filelist[0]), binary_backend="jpg")
fig.update_layout(
    newshape_line_color=color_dict[DEFAULT_ATYPE],
    margin=dict(l=0, r=0, b=0, t=0, pad=4),
    dragmode="drawrect",
)

# Buttons
button_gh = dbc.Button(
    "Learn more",
    id="howto-open",
    outline=True,
    color="secondary",
    # Turn off lowercase transformation for class .button in stylesheet
    style={"textTransform": "none"},
)

button_howto = dbc.Button(
    "View Code on github",
    outline=True,
    color="primary",
    href="https://github.com/plotly/dash-sample-apps/tree/master/apps/dash-image-annotation",
    id="gh-link",
    style={"text-transform": "none"},
)

# Modal
modal_overlay = dbc.Modal(
    [
        dbc.ModalBody(html.Div([dcc.Markdown(howto, id="howto-md")])),
        dbc.ModalFooter(dbc.Button("Close", id="howto-close", className="howto-bn",)),
    ],
    id="modal",
    size="lg",
    style={"font-size": "small"},
)

# Cards
image_annotation_card = dbc.Card(
    id="imagebox",
    children=[
        dbc.CardHeader(html.H2("Annotation area")),
        dbc.CardBody(
            [
                dcc.Graph(
                    id="graph",
                    figure=fig,
                    config={"modeBarButtonsToAdd": ["drawrect", "eraseshape"]},
                )
            ]
        ),
        dbc.CardFooter(
            [
                dcc.Markdown(
                    "To annotate the above image, select an appropriate label on the right and then draw a "
                    "rectangle with your cursor around the area of the image you wish to annotate.\n\n"
                    "**Choose a different image to annotate**:"
                ),
                dbc.ButtonGroup(
                    [
                        dbc.Button("Previous image", id="previous", outline=True),
                        dbc.Button("Next image", id="next", outline=True),
                    ],
                    size="lg",
                    style={"width": "100%"},
                ),
            ]
        ),
    ],
)

annotated_data_card = dbc.Card(
    [
        dbc.CardHeader(html.H2("Annotated data")),
        dbc.CardBody(
            [
                dbc.Row(dbc.Col(html.H3("Coordinates of annotations"))),
                dbc.Row(
                    dbc.Col(
                        [
                            dash_table.DataTable(
                                id="annotations-table",
                                columns=[
                                    dict(
                                        name=n,
                                        id=n,
                                        presentation=(
                                            "dropdown" if n == "Type" else "input"
                                        ),
                                    )
                                    for n in columns
                                ],
                                editable=True,
                                style_data={"height": 40},
                                style_cell={
                                    "overflow": "hidden",
                                    "textOverflow": "ellipsis",
                                    "maxWidth": 0,
                                },
                                dropdown={
                                    "Type": {
                                        "options": [
                                            {"label": o, "value": o}
                                            for o in annotation_types
                                        ],
                                        "clearable": False,
                                    }
                                },
                                style_cell_conditional=[
                                    {"if": {"column_id": "Type"}, "textAlign": "left",}
                                ],
                                fill_width=True,
                            ),
                            dcc.Store(id="graph-copy", data=fig),
                            dcc.Store(
                                id="annotations-store",
                                data=dict(
                                    **{
                                        filename: {"shapes": []}
                                        for filename in filelist
                                    },
                                    **{"starttime": time_passed()}
                                ),
                            ),
                            dcc.Store(
                                id="image_files",
                                data={"files": filelist, "current": 0},
                            ),
                        ],
                    ),
                ),
                dbc.Row(
                    dbc.Col(
                        [
                            html.H3("Create new annotation for"),
                            dcc.Dropdown(
                                id="annotation-type-dropdown",
                                options=[
                                    {"label": t, "value": t} for t in annotation_types
                                ],
                                value=DEFAULT_ATYPE,
                                clearable=False,
                            ),
                        ],
                        align="center",
                    )
                ),
            ]
        ),
        dbc.CardFooter(
            [
                html.Div(
                    [
                        # We use this pattern because we want to be able to download the
                        # annotations by clicking on a button
                        html.A(
                            id="download",
                            download="annotations.json",
                            # make invisble, we just want it to click on it
                            style={"display": "none"},
                        ),
                        dbc.Button(
                            "Download annotations", id="download-button", outline=True,
                        ),
                        html.Div(id="dummy", style={"display": "none"}),
                        dbc.Tooltip(
                            "You can download the annotated data in a .json format by clicking this button",
                            target="download-button",
                        ),
                    ],
                )
            ]
        ),
    ],
)

# Navbar
navbar = dbc.Navbar(
    dbc.Container(
        [
            dbc.Row(
                [
                    dbc.Col(
                        html.A(
                            html.Img(
                                src=app.get_asset_url("dash-logo-new.png"),
                                height="30px",
                            ),
                            href="https://plot.ly",
                        )
                    ),
                    dbc.Col(dbc.NavbarBrand("Image Annotation App")),
                ],
                align="center",
            ),
            dbc.Row(
                dbc.Col(
                    [
                        dbc.NavbarToggler(id="navbar-toggler"),
                        dbc.Collapse(
                            dbc.Nav(
                                [dbc.NavItem(button_howto), dbc.NavItem(button_gh)],
                                className="ml-auto",
                                navbar=True,
                            ),
                            id="navbar-collapse",
                            navbar=True,
                        ),
                        modal_overlay,
                    ]
                ),
                align="center",
            ),
        ],
        fluid=True,
    ),
    color="dark",
    dark=True,
    className="mb-5",
)

app.layout = html.Div(
    [
        navbar,
        dbc.Container(
            [
                dbc.Row(
                    [
                        dbc.Col(image_annotation_card, md=7),
                        dbc.Col(annotated_data_card, md=5),
                    ],
                ),
            ],
            fluid=True,
        ),
    ]
)


@app.callback(
    [Output("annotations-table", "data"), Output("image_files", "data")],
    [
        Input("previous", "n_clicks"),
        Input("next", "n_clicks"),
        Input("graph", "relayoutData"),
    ],
    [
        State("annotations-table", "data"),
        State("image_files", "data"),
        State("annotations-store", "data"),
        State("annotation-type-dropdown", "value"),
    ],
)
def modify_table_entries(
    previous_n_clicks,
    next_n_clicks,
    graph_relayoutData,
    annotations_table_data,
    image_files_data,
    annotations_store_data,
    annotation_type,
):
    cbcontext = [p["prop_id"] for p in dash.callback_context.triggered][0]
    if cbcontext == "graph.relayoutData":
        debug_print("graph_relayoutData:", graph_relayoutData)
        debug_print("annotations_table_data before:", annotations_table_data)
        if "shapes" in graph_relayoutData.keys():
            # this means all the shapes have been passed to this function via
            # graph_relayoutData, so we store them
            annotations_table_data = [
                shape_to_table_row(sh) for sh in graph_relayoutData["shapes"]
            ]
        elif re.match("shapes\[[0-9]+\].x0", list(graph_relayoutData.keys())[0]):
            # this means a shape was updated (e.g., by clicking and dragging its
            # vertices), so we just update the specific shape
            annotations_table_data = annotations_table_shape_resize(
                annotations_table_data, graph_relayoutData
            )
        if annotations_table_data is None:
            return dash.no_update
        else:
            debug_print("annotations_table_data after:", annotations_table_data)
            return (annotations_table_data, image_files_data)
    image_index_change = 0
    if cbcontext == "previous.n_clicks":
        image_index_change = -1
    if cbcontext == "next.n_clicks":
        image_index_change = 1
    image_files_data["current"] += image_index_change
    image_files_data["current"] %= len(image_files_data["files"])
    if image_index_change != 0:
        # image changed, update annotations_table_data with new data
        annotations_table_data = []
        filename = image_files_data["files"][image_files_data["current"]]
        debug_print(annotations_store_data[filename])
        for sh in annotations_store_data[filename]["shapes"]:
            annotations_table_data.append(shape_to_table_row(sh))
        return (annotations_table_data, image_files_data)
    else:
        return dash.no_update


@app.callback(
    [Output("graph", "figure"), Output("annotations-store", "data"),],
    [Input("annotations-table", "data"), Input("annotation-type-dropdown", "value")],
    [State("image_files", "data"), State("annotations-store", "data")],
)
def send_figure_to_graph(
    annotations_table_data, annotation_type, image_files_data, annotations_store
):
    if annotations_table_data is not None:
        filename = image_files_data["files"][image_files_data["current"]]
        # convert table rows to those understood by fig.update_layout
        fig_shapes = [table_row_to_shape(sh) for sh in annotations_table_data]
        debug_print("fig_shapes:", fig_shapes)
        debug_print(
            "annotations_store[%s]['shapes']:" % (filename,),
            annotations_store[filename]["shapes"],
        )
        # find the shapes that are new
        new_shapes_i = []
        old_shapes_i = []
        for i, sh in enumerate(fig_shapes):
            if not shape_in(annotations_store[filename]["shapes"])(sh):
                new_shapes_i.append(i)
            else:
                old_shapes_i.append(i)
        # add timestamps to the new shapes
        for i in new_shapes_i:
            fig_shapes[i]["timestamp"] = time_passed(annotations_store["starttime"])
        # find the old shapes and look up their timestamps
        for i in old_shapes_i:
            old_shape_i = index_of_shape(
                annotations_store[filename]["shapes"], fig_shapes[i]
            )
            fig_shapes[i]["timestamp"] = annotations_store[filename]["shapes"][
                old_shape_i
            ]["timestamp"]
        shapes = fig_shapes
        debug_print("shapes:", shapes)
        fig = px.imshow(io.imread(filename), binary_backend="jpg")
        fig.update_layout(
            shapes=[shape_data_remove_timestamp(sh) for sh in shapes],
            # reduce space between image and graph edges
            newshape_line_color=color_dict[annotation_type],
            margin=dict(l=0, r=0, b=0, t=0, pad=4),
            dragmode="drawrect",
        )
        annotations_store[filename]["shapes"] = shapes
        return (fig, annotations_store)
    return dash.no_update


@app.callback(
    Output("modal", "is_open"),
    [Input("howto-open", "n_clicks"), Input("howto-close", "n_clicks")],
    [State("modal", "is_open")],
)
def toggle_modal(n1, n2, is_open):
    if n1 or n2:
        return not is_open
    return is_open


# set the download url to the contents of the annotations-store (so they can be
# downloaded from the browser's memory)
app.clientside_callback(
    """
function(the_store_data) {
    let s = JSON.stringify(the_store_data);
    let b = new Blob([s],{type: 'text/plain'});
    let url = URL.createObjectURL(b);
    return url;
}
""",
    Output("download", "href"),
    [Input("annotations-store", "data")],
)

# click on download link via button
app.clientside_callback(
    """
function(download_button_n_clicks)
{
    let download_a=document.getElementById("download");
    download_a.click();
    return '';
}
""",
    Output("dummy", "children"),
    [Input("download-button", "n_clicks")],
)


# TODO comment the dbc link
# we use a callback to toggle the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


if __name__ == "__main__":
    app.run_server()

### Chat GPT:

This code appears to be a part of a Dash application for image annotation. Let's go through the code step by step:

The code starts by importing the required libraries for building the Dash application, including dash, dash_html_components, dash_core_components, dash_bootstrap_components, dash_table, plotly.express, and skimage.io.

The code defines some constants and variables used in the application, such as DEBUG (a boolean flag for debugging), NUM_ATYPES (number of annotation types), DEFAULT_FIG_MODE (default figure mode), annotation_colormap (a colormap for annotation types), annotation_types (a list of annotation types), and DEFAULT_ATYPE (default annotation type).

The code prepares a bijective type<->color mapping using the typ_col_pairs list and creates two dictionaries (color_dict and type_dict) to map types to colors and colors to types, respectively.

The code defines some utility functions:

debug_print(*args): A function to print debug messages if DEBUG is True.
coord_to_tab_column(coord): A function to convert a coordinate to a table column name.
time_passed(start=0): A function to calculate the time passed since a given start time.
format_float(f): A function to format a float number with two decimal places.
shape_to_table_row(sh): A function to convert a shape dictionary to a table row dictionary.
default_table_row(): A function to create a default table row dictionary.
table_row_to_shape(tr): A function to convert a table row dictionary to a shape dictionary.
shape_cmp(s0, s1): A function to compare two shape dictionaries.
shape_in(se): A function to check if a shape is in a list of shapes.
index_of_shape(shapes, shape): A function to find the index of a shape in a list of shapes.
annotations_table_shape_resize(annotations_table_data, fig_data): A function to extract the shape that was resized and update the corresponding coordinates in the annotations table data.
shape_data_remove_timestamp(shape): A function to remove the "timestamp" key from a shape dictionary.
The code sets up the Dash application by creating an instance of the Dash class, specifying the external stylesheets and creating the server instance.

It defines the layout of the application using Dash's HTML and Bootstrap components. The layout includes a navbar, two cards for image annotation and annotated data, and some buttons and tooltips.

The code defines several callback functions using the app.callback decorator. These callbacks are used to update different components of the application based on user interactions or changes in the data.

modify_table_entries: This callback function is triggered by the "Previous", "Next", or "graph.relayoutData" inputs. It handles the logic for updating the annotations table data and the current image file data based on user interactions with the application. It returns the updated table data and image files data.

send_figure_to_graph: This callback function is triggered by the "annotations-table.data" or "annotation-type-dropdown.value" inputs. It handles the logic for updating the figure and the annotations store data based on changes in the annotations table data or the annotation type selected by the user. It returns the updated figure and annotations store data.

Finally, the code sets the app.layout to the main layout defined earlier.

The code seems to be part of a larger application that allows users to annotate

### User:

Modify the following so that the user can upload their own image to be annotated upon

### Chat GPT:

To allow users to upload their own image for annotation, you can modify the code as follows:

Import the dash_upload_components library at the beginning of the code to handle file uploads. You can install this library using pip install dash-upload-components.

In [ ]:
import dash_upload_components as duc

Add a new Upload component to the layout section of the code. This component will allow users to upload their image file.

In [ ]:
app.layout = dbc.Container(
    [
        dbc.Navbar(...),
        html.Div(
            [
                dbc.Card(
                    [
                        ...
                    ],
                    ...
                ),
                dbc.Card(
                    [
                        dbc.CardHeader("Annotated Data"),
                        dbc.CardBody(
                            [
                                ...
                            ]
                        ),
                    ],
                    ...
                ),
            ],
            ...
        ),
        duc.Upload(
            id="upload-image",
            max_files=1,
            filetypes=["image/*"],
            text="Drag and drop or click to upload an image",
        ),
    ],
    ...
)


Define a new callback function upload_image that will handle the image upload event and update the application state with the uploaded image.

In [ ]:
@app.callback(
    Output("image", "src"),
    Output("annotations-table", "data"),
    Input("upload-image", "contents"),
)
def upload_image(contents):
    if contents is not None:
        # Decode the uploaded image
        content_type, content_string = contents[0].split(",")
        decoded_image = base64.b64decode(content_string)

        # Process the image and update the application state
        # TODO: Add your image processing logic here

        # Return the updated image source and annotations table data
        return dcc.Location.pathname, "data:image/png;base64," + base64.b64encode(decoded_image).decode(), []

    # If no image is uploaded, return the default image source and annotations table data
    return dcc.Location.pathname, "/assets/default_image.png", []


In the modify_table_entries callback function, update the logic to handle the case when no image is uploaded by the user. If no image is uploaded, return the default image source and an empty annotations table data.

In [ ]:
@app.callback(
    Output("annotations-table", "data"),
    Output("current-image-file", "data"),
    Input("previous-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    Input("graph", "relayoutData"),
)
def modify_table_entries(previous_clicks, next_clicks, relayout_data):
    # Check if no image is uploaded
    if current_image_file is None:
        return []

    # Existing logic for modifying table entries
    ...


Update the send_figure_to_graph callback function to handle the case when no image is uploaded. If no image is uploaded, return the default figure and an empty annotations store data.

In [ ]:
@app.callback(
    Output("graph", "figure"),
    Output("annotations-store", "data"),
    Input("annotations-table", "data"),
    Input("annotation-type-dropdown", "value"),
)
def send_figure_to_graph(table_data, annotation_type):
    # Check if no image is uploaded
    if current_image_file is None:
        return go.Figure(), []

    # Existing logic for sending figure to graph
    ...


Remember to add any required import statements and modify any other parts of the code that may depend on the image upload feature. Additionally, you'll need to implement the image processing logic in the upload_image callback to handle the uploaded image and update the application state accordingly.